In [8]:
from pyomo.environ import *
import numpy as np
import pandas as pd

In [9]:
proj_dir =  "ann/frac3/"
ann_params = {}
ann_params["fc1_weight"] = pd.read_csv(proj_dir + "fc1.weight.csv", header=None, index_col=False).to_numpy().T
ann_params["fc2_weight"] = pd.read_csv(proj_dir + "fc2.weight.csv", header=None, index_col=False).to_numpy().T
ann_params["fc3_weight"] = pd.read_csv(proj_dir + "fc3.weight.csv", header=None, index_col=False).to_numpy().T
ann_params["fc4_weight"] = pd.read_csv(proj_dir + "fc4.weight.csv", header=None, index_col=False).to_numpy().T

ann_params["fc1_bias"] = pd.read_csv(proj_dir + "fc1.bias.csv", header=None, index_col=False).to_numpy().flatten()
ann_params["fc2_bias"] = pd.read_csv(proj_dir + "fc2.bias.csv", header=None, index_col=False).to_numpy().flatten()
ann_params["fc3_bias"] = pd.read_csv(proj_dir + "fc3.bias.csv", header=None, index_col=False).to_numpy().flatten()
ann_params["fc4_bias"] = pd.read_csv(proj_dir + "fc4.bias.csv", header=None, index_col=False).to_numpy().flatten()

In [10]:
for key, val in ann_params.items():
    print(f"{key} : {np.shape(val)}")

fc1_weight : (2, 15)
fc2_weight : (15, 18)
fc3_weight : (18, 12)
fc4_weight : (12, 3)
fc1_bias : (15,)
fc2_bias : (18,)
fc3_bias : (12,)
fc4_bias : (3,)
fc1_weight : (2, 15)
fc2_weight : (15, 18)
fc3_weight : (18, 12)
fc4_weight : (12, 3)
fc1_bias : (15,)
fc2_bias : (18,)
fc3_bias : (12,)
fc4_bias : (3,)


In [11]:
architecture = {
        'input': 2,
        'fc1': 15,
        'fc2': 18,
        'fc3': 12,
        'fc4': 3
    }

def setup_pyomo_model_from_ann2(m, ann_pars, weight_mapper, bias_mapper):
    """
    Sets up a Pyomo model based on neural network parameters.

    Args:
        model (ConcreteModel): An instance of Pyomo ConcreteModel.
        parameters (dict): A dictionary of neural network weights and biases.
        surrogate_index (int): An index to distinguish between different surrogate models.
    """

    m.neuro2_input = Set(initialize=range(2))
    m.neuro2_l1 = Set(initialize=range(15))
    m.neuro2_l2 = Set(initialize=range(18))
    m.neuro2_l3 = Set(initialize=range(12))
    m.neuro2_l4 = Set(initialize=range(3))

    m.M2 = Param(initialize=1e12)

    def _w2_l0_rule(m, n, nk):
        return ann_pars[weight_mapper[0]][n][nk]

    def _w2_l1_rule(m, n, nk):
        return ann_pars[weight_mapper[1]][n][nk]

    def _w2_l2_rule(m, n, nk):
        return ann_pars[weight_mapper[2]][n][nk]

    def _w2_l3_rule(m, n, nk):
        return ann_pars[weight_mapper[3]][n][nk]

    m.w2_l0 = Param(m.neuro2_input, m.neuro2_l1, initialize=_w2_l0_rule)
    m.w2_l1 = Param(m.neuro2_l1, m.neuro2_l2, initialize=_w2_l1_rule)
    m.w2_l2 = Param(m.neuro2_l2, m.neuro2_l3, initialize=_w2_l2_rule)
    m.w2_l3 = Param(m.neuro2_l3, m.neuro2_l4, initialize=_w2_l3_rule)

    def _b2_l0_rule(m, n):
        return ann_pars[bias_mapper[0]][n]

    def _b2_l1_rule(m, n):
        return ann_pars[bias_mapper[1]][n]

    def _b2_l2_rule(m, n):
        return ann_pars[bias_mapper[2]][n]

    def _b2_l3_rule(m, n):
        return ann_pars[bias_mapper[3]][n]

    m.b2_l0 = Param(m.neuro2_l1, initialize=_b2_l0_rule)
    m.b2_l1 = Param(m.neuro2_l2, initialize=_b2_l1_rule)
    m.b2_l2 = Param(m.neuro2_l3, initialize=_b2_l2_rule)
    m.b2_l3 = Param(m.neuro2_l4, initialize=_b2_l3_rule)

    m.z2_l0 = Var(m.neuro2_input, domain=Reals)
    m.z2_l1 = Var(m.neuro2_l1, domain=Reals)
    m.z2_l2 = Var(m.neuro2_l2, domain=Reals)
    m.z2_l3 = Var(m.neuro2_l3, domain=Reals)
    m.z2_l4 = Var(m.neuro2_l4, domain=Reals)

    m.a2_l1 = Var(m.neuro2_l1, domain=Reals)
    m.a2_l2 = Var(m.neuro2_l2, domain=Reals)
    m.a2_l3 = Var(m.neuro2_l3, domain=Reals)
    m.a2_l4 = Var(m.neuro2_l4, domain=Reals)

    m.d2_l1 = Var(m.neuro2_l1, domain=Binary)
    m.d2_l2 = Var(m.neuro2_l2, domain=Binary)
    m.d2_l3 = Var(m.neuro2_l3, domain=Binary)
    # m.d2_l4 = Var(m.neuro2_l4, domain=Binary)

    # ============== layer 1 =================

    def sur2_a2_l1(m, n):
        return m.a2_l1[n] == sum(m.w2_l0[nprev, n] * m.z2_l0[nprev] for nprev in m.neuro2_input) + m.b2_l0[n]
    m.sur2_a2_l1 = Constraint(m.neuro2_l1, rule=sur2_a2_l1)

    def sur2_A2_l1(m, n):
        return -m.M2 * (1 - m.d2_l1[n]) <= m.a2_l1[n]
    m.sur2_A2_l1 = Constraint(m.neuro2_l1, rule=sur2_A2_l1)

    def sur2_A3_l1(m, n):
        return m.a2_l1[n] <= m.M2 * m.d2_l1[n]
    m.sur2_A3_l1 = Constraint(m.neuro2_l1, rule=sur2_A3_l1)

    def sur2_A4a_l1(m, n):
        return 0 <= m.z2_l1[n]
    m.sur2_A4a_l1 = Constraint(m.neuro2_l1, rule=sur2_A4a_l1)

    def sur2_A4b_l1(m, n):
        return m.z2_l1[n] <=  m.M2 * m.d2_l1[n]
    m.sur2_A4b_l1 = Constraint(m.neuro2_l1, rule=sur2_A4b_l1)

    def sur2_A5a_l1(m, n):
        return m.a2_l1[n] - m.M2 * (1 - m.d2_l1[n]) <= m.z2_l1[n]
    m.sur2_A5a_l1 = Constraint(m.neuro2_l1, rule=sur2_A5a_l1)

    def sur2_A5b_l1(m, n):
        return m.z2_l1[n] <= m.a2_l1[n] + m.M2 * (1 - m.d2_l1[n])
    m.sur2_A5b_l1 = Constraint(m.neuro2_l1, rule=sur2_A5b_l1)

    # ============== layer 2 =================

    def sur2_a2_l2(m, n):
        return m.a2_l2[n] == sum(m.w2_l1[nprev, n] * m.z2_l1[nprev] for nprev in m.neuro2_l1) + m.b2_l1[n]
    m.sur2_a2_l2 = Constraint(m.neuro2_l2, rule=sur2_a2_l2)

    def sur2_A2_l2(m, n):
        return -m.M2 * (1 - m.d2_l2[n]) <= m.a2_l2[n]
    m.sur2_A2_l2 = Constraint(m.neuro2_l2, rule=sur2_A2_l2)

    def sur2_A3_l2(m, n):
        return m.a2_l2[n] <= m.M2 * m.d2_l2[n]
    m.sur2_A3_l2 = Constraint(m.neuro2_l2, rule=sur2_A3_l2)

    def sur2_A4a_l2(m, n):
        return 0 <= m.z2_l2[n]
    m.sur2_A4a_l2 = Constraint(m.neuro2_l2, rule=sur2_A4a_l2)

    def sur2_A4b_l2(m, n):
        return m.z2_l2[n] <=  m.M2 * m.d2_l2[n]
    m.sur2_A4b_l2 = Constraint(m.neuro2_l2, rule=sur2_A4b_l2)

    def sur2_A5a_l2(m, n):
        return m.a2_l2[n] - m.M2 * (1 - m.d2_l2[n]) <= m.z2_l2[n]
    m.sur2_A5a_l2 = Constraint(m.neuro2_l2, rule=sur2_A5a_l2)

    def sur2_A5b_l2(m, n):
        return m.z2_l2[n] <= m.a2_l2[n] + m.M2 * (1 - m.d2_l2[n])
    m.sur2_A5b_l2 = Constraint(m.neuro2_l2, rule=sur2_A5b_l2)

    # ============== layer 3 =================

    def sur2_a2_l3(m, n):
        return m.a2_l3[n] == sum(m.w2_l2[nprev, n] * m.z2_l2[nprev] for nprev in m.neuro2_l2) + m.b2_l2[n]
    m.sur2_a2_l3 = Constraint(m.neuro2_l3, rule=sur2_a2_l3)

    def sur2_A2_l3(m, n):
        return -m.M2 * (1 - m.d2_l3[n]) <= m.a2_l3[n]
    m.sur2_A2_l3 = Constraint(m.neuro2_l3, rule=sur2_A2_l3)

    def sur2_A3_l3(m, n):
        return m.a2_l3[n] <= m.M2 * m.d2_l3[n]
    m.sur2_A3_l3 = Constraint(m.neuro2_l3, rule=sur2_A3_l3)

    def sur2_A4a_l3(m, n):
        return 0 <= m.z2_l3[n]
    m.sur2_A4a_l3 = Constraint(m.neuro2_l3, rule=sur2_A4a_l3)

    def sur2_A4b_l3(m, n):
        return m.z2_l3[n] <=  m.M2 * m.d2_l3[n]
    m.sur2_A4b_l3 = Constraint(m.neuro2_l3, rule=sur2_A4b_l3)

    def sur2_A5a_l3(m, n):
        return m.a2_l3[n] - m.M2 * (1 - m.d2_l3[n]) <= m.z2_l3[n]
    m.sur2_A5a_l3 = Constraint(m.neuro2_l3, rule=sur2_A5a_l3)

    def sur2_A5b_l3(m, n):
        return m.z2_l3[n] <= m.a2_l3[n] + m.M2 * (1 - m.d2_l3[n])
    m.sur2_A5b_l3 = Constraint(m.neuro2_l3, rule=sur2_A5b_l3)

     # ============== layer 4 =================

    def sur2_a2_l4(m, n):
        return m.a2_l4[n] == sum(m.w2_l3[nprev, n] * m.z2_l3[nprev] for nprev in m.neuro2_l3) + m.b2_l3[n]
    m.sur2_a2_l4 = Constraint(m.neuro2_l4, rule=sur2_a2_l4)

    def sur2_A6_l4(m, n):
        return m.a2_l4[n] == m.z2_l4[n]
    m.sur2_A6_l4 = Constraint(m.neuro2_l4, rule=sur2_A6_l4)

    return




In [12]:
ann_weight_mapper = {0: "fc1_weight", 1: "fc2_weight", 2: "fc3_weight", 3: "fc4_weight"}
ann_bias_mapper = {0: "fc1_bias", 1: "fc2_bias", 2: "fc3_bias", 3: "fc4_bias"}

model = ConcreteModel()
setup_pyomo_model_from_ann2(model, ann_params, ann_weight_mapper, ann_bias_mapper)

In [13]:
# model.pprint();

9 Set Declarations
    neuro2_input : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}
    neuro2_l1 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   15 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
    neuro2_l2 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   18 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
    neuro2_l3 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    neuro2_l4 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    w2_l0_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                 : Size : Me

In [14]:
model.z2_l0[0].fix(0)
model.z2_l0[1].fix(0)


def solve_model(m):
    m.obj = Objective(expr=model.z2_l4[1] + model.z2_l4[2], sense=minimize)

    opt1 = SolverFactory('gams')
    io_options = dict()

    io_options['solver'] = "cplex"
    res = opt1.solve(m,
        tee=True,
        io_options=io_options)

    return res

solve_model(model)
model.z2_l4.pprint()

--- Job model.gms Start 12/13/23 11:14:21 43.2.0 859d62d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\43\gmsprmNT.txt
    C:\Users\justi\OneDrive\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\justi\AppData\Local\Temp\tmpygsxq296\model.gms
    Output C:\Users\justi\AppData\Local\Temp\tmpygsxq296\output.lst
    ScrDir C:\Users\justi\AppData\Local\Temp\tmpygsxq296\225a\
    SysDir C:\GAMS\43\
    CurDir C:\Users\justi\AppData\Local\Temp\tmpygsxq296\
    LogOption 3
Licensee: Small MUD - 5 User License                     G221121|0002AP-GEN
          University of Delaware, Chemical and Biomolecular EngineeriDC3967
          C:\GAMS\43\gamslice.txt
          License Admin: Marianthi Ierapetritou, mgi@udel.edu              
Processor information: 1 socket(s), 10 core(s), and 20 thread(s) available
GAMS 43.2.0   Copyright (C) 1987-2023 GAMS Development. All rights reserved
--- Starting compilation
--- model.gms(1322) 2 Mb
--- Starting execution: ela